In [1]:
from CHPA import *

engine = create_engine("mssql+pymssql://(local)/CHPA_1806")
table_name = "data"

In [2]:
condition = (
    "[TC III] in ('C09A ACE INHIBITORS PLAIN|血管紧张素转换酶抑制剂，单一用药' ,"
    "'C09B ACE INHIBITORS COMBS|血管紧张素转换酶抑制剂，联合用药' ,"
    "'C09C ANGIOTENS-II ANTAG, PLAIN|血管紧张素II拮抗剂，单一用药' ,"
    "'C09D ANGIOTEN-II ANTAG, COMB|血管紧张素II拮抗剂，联合用药') and MOLECULE != '沙库巴曲,缬沙坦|SACUBITRIL+VALSARTAN'"
)

sql = "SELECT * FROM " + table_name + " WHERE " + condition
df = pd.read_sql(sql=sql, con=engine)

In [6]:
df["MOLECULE"] = df["MOLECULE"].str.split("|").str[0]
df["PRODUCT"] = (
    df["PRODUCT"].str.split("|").str[0]
)  # + '（'+ df['PRODUCT'].str.split('|').str[1].str[-3:] +'）'
df["PRODUCT_CORP"] = (
    df["PRODUCT_CORP"].str.split("（").str[0].str.split("|").str[0]
    + "\n"
    + df["PRODUCT_CORP"].str.split("（").str[1].str.split("|").str[0]
)
mask = df["MOLECULE"] == "氯沙坦钾"
df.loc[mask, "MOLECULE"] = "氯沙坦"
mask = df["MOLECULE"] == "培哚普利叔丁胺"
df.loc[mask, "MOLECULE"] = "培哚普利"
mask = df["MOLECULE"].isin(["氨氯地平贝那普利(II)", "氨氯地平贝那普利"])
df.loc[mask, "MOLECULE"] = "贝那普利,氨氯地平"
mask = df["MOLECULE"].isin(
    ["厄贝沙坦", "缬沙坦", "氯沙坦", "奥美沙坦", "坎地沙坦", "厄贝沙坦,氢氯噻嗪", "福辛普利", "卡托普利", "赖诺普利", "依那普利"]
)
df.loc[mask, "VBP"] = "带量品种"
mask = (
    df["MOLECULE"].isin(
        [
            "厄贝沙坦",
            "缬沙坦",
            "氯沙坦",
            "奥美沙坦",
            "坎地沙坦",
            "厄贝沙坦,氢氯噻嗪",
            "福辛普利",
            "卡托普利",
            "赖诺普利",
            "依那普利",
            "培哚普利",
            "替米沙坦" "缬沙坦,氨氯地平",
            "缬沙坦,氢氯噻嗪",
        ]
    )
    == False
)
df.loc[mask, "VBP"] = "非带量品种"

mask = df["UNIT"] == "Volume (Counting Unit)"
df_std_volume = df.loc[mask, :]
df_std_volume["UNIT"] = "Volume (Std Counting Unit)"
df = pd.concat([df, df_std_volume])

convert_std_volume(df, "MOLECULE", "氯沙坦", "100MG", 2)
convert_std_volume(df, "MOLECULE", "厄贝沙坦", "75MG", 0.5)
convert_std_volume(df, "MOLECULE", "替米沙坦", "20MG", 0.25)
convert_std_volume(df, "MOLECULE", "替米沙坦", "40MG", 0.5)
convert_std_volume(df, "MOLECULE", "坎地沙坦", "4MG", 0.5)
convert_std_volume(df, "MOLECULE", "缬沙坦", "40MG", 0.5)
convert_std_volume(df, "MOLECULE", "缬沙坦", "160MG", 2)
convert_std_volume(df, "MOLECULE", "培哚普利", "8MG", 2)
convert_std_volume(df, "MOLECULE", "贝那普利", "5MG", 0.5)
convert_std_volume(df, "MOLECULE", "卡托普利", "12.5MG", 0.25)
convert_std_volume(df, "MOLECULE", "卡托普利", "25MG", 0.5)
convert_std_volume(df, "MOLECULE", "依那普利", "5MG", 0.5)
convert_std_volume(df, "MOLECULE", "雷米普利", "2.5MG", 0.5)

mask = df["MOLECULE"].str.contains("普利")
df.loc[mask, "TC III"] = "ACEI"
mask = df["MOLECULE"].str.contains("沙坦")
df.loc[mask, "TC III"] = "ARB"
mask = df["MOLECULE"].str.contains("地平")
df.loc[mask, "TC III"] = "A+C"
mask = df["MOLECULE"].str.contains("噻嗪|舒脈康膜衣錠|叶酸|吲达帕胺|氢噻")
df.loc[mask, "TC III"] = "A+D"

AttributeError: Can only use .str accessor with string values!

In [4]:
r = chpa(df, name="ARB（单方+复方）定义市场")

r.plot_overall_performance(dimension='TC III')
r.plot_overall_performance(dimension='TC III', unit="Volume (Std Counting Unit)")

                    ACEI           ARB      RAAS FDC
DATE                                                
2017-06-01  1.472503e+09  5.347627e+09  2.979592e+09
2017-09-01  1.495925e+09  5.436922e+09  3.068514e+09
2017-12-01  1.521684e+09  5.491926e+09  3.152960e+09
2018-03-01  1.526744e+09  5.575126e+09  3.258798e+09
2018-06-01  1.526497e+09  5.647480e+09  3.354287e+09
2018-09-01  1.550908e+09  5.732649e+09  3.460444e+09
2018-12-01  1.546155e+09  5.840139e+09  3.582233e+09
2019-03-01  1.594897e+09  6.113838e+09  3.810068e+09
2019-06-01  1.644511e+09  6.352595e+09  4.070412e+09
2019-09-01  1.676089e+09  6.591193e+09  4.374896e+09
2019-12-01  1.696357e+09  6.692869e+09  4.602839e+09
2020-03-01  1.628390e+09  6.383159e+09  4.642838e+09
2020-06-01  1.573417e+09  5.929519e+09  4.750539e+09
2020-09-01  1.532034e+09  5.408139e+09  4.927745e+09
2020-12-01  1.508899e+09  4.818678e+09  5.179880e+09
2021-03-01  1.509538e+09  4.303712e+09  5.702953e+09
2021-06-01  1.448130e+09  3.893524e+09  6.1470